### Creating Training Data for UFC Fight Predictions  

This notebook focuses on building a **clean and structured training dataset** for UFC fight predictions by integrating multiple data sources. The main steps include:

##### **1. Constructing the Baseline Dataset**  
   - Generate a **unique bout dataset**, ensuring each fight is represented only once.  
   - Standardize fighter names, event details, and match outcomes.  

##### **2. Creating a Fighter Statistics Dataset**  
   - Extract **fighter-level statistics** for each available year.  
   - Compute **historical performance metrics** such as win streaks, win percentages, and fight history summaries.  

##### **3. Merging and Aligning the Data**  
   - **Cross-join events by date** to match fight records across different data sources.  
   - Apply **string matching techniques (token set ratio)** to align event names with high accuracy.  
   - Use **fighter-opponent string similarity** to ensure correct fight mappings.  
   - Merge **fighter statistics** with fight-level data to enrich the dataset.  

#### **4. Feature Engineering for Model Training**  
   - Compute **rolling averages** of fight statistics to capture historical trends.  
   - Extract **physical attributes (height, reach, age)** from fighter profiles.  
   - Ensure **no data leakage** by adjusting win/loss and rolling averages columns to reflect only prior information.  

The resulting dataset serves as a **training-ready dataset**, combining Wikipedia fight records with UFC statistics. This structured data can be used to train predictive models, such as machine learning classifiers, to forecast fight outcomes. 🚀  

In [48]:
import pandas as pd
import json
import os
import re
from thefuzz import fuzz

In [49]:
# get the file path
def get_file_path(folder, file_name):
    return os.path.dirname(os.getcwd()) + "/" + folder + "/" + file_name

### Load all files 

In [50]:
# read json file function
def read_json_file(folder="collection/raw", json_file=None):
    path = get_file_path(folder, json_file)
    with open(path, "r") as file:
        data = [json.loads(line) for line in file]
    return pd.DataFrame(data)

# Data extracted from ufc-stats-website - (project name: ufc_stats_spider)
event_data = read_json_file(json_file="event.json")
event_card_data = read_json_file(json_file="event_card.json")
fight_details_data = read_json_file(json_file="fight_details.json")
fight_results_data = read_json_file(json_file="fight_results.json")
fighter_details = read_json_file(json_file="fighter_details.json")


# Data extracted from wikipedia - (project name: octa-data)
record_table = pd.read_csv(get_file_path("collection/raw", "wikipedia_record_table.csv"))

# print head of the data
print("Event Data: ",event_data.head())
print("Event Card Data: ",event_card_data.head())
print("Fight Details Data: ",fight_details_data.head())
print("Fight Results Data: ",fight_results_data.head())
print("Fighter Details Data: ",fighter_details.head())

Event Data:      type                                          event_url  \
0  event  http://www.ufcstats.com/event-details/9ca265df...   
1  event  http://www.ufcstats.com/event-details/063649e2...   
2  event  http://www.ufcstats.com/event-details/18f5669a...   
3  event  http://www.ufcstats.com/event-details/956dcac6...   
4  event  http://www.ufcstats.com/event-details/3cf68c1d...   

                                event_name  event_date  \
0              UFC 173: Barao vs Dillashaw  2014-05-24   
1  UFC Fight Night: Henderson vs Dos Anjos  2014-08-23   
2          UFC Fight Night: Kattar vs. Ige  2020-07-15   
3                                 DWCS 6.3  2022-08-09   
4          UFC 243: Whittaker vs. Adesanya  2019-10-05   

                               event_location    event_key  
0                      Las Vegas, Nevada, USA  67220140524  
1                        Tulsa, Oklahoma, USA  23820140823  
2  Abu Dhabi, Abu Dhabi, United Arab Emirates  12720200715  
3              

###  Data pre-processing steps  

In [51]:
# Convert columns to string
record_table.loc[:,'Event'] = record_table.Event.astype(str)
event_data.loc[:,'event_name'] = event_data.event_name.astype(str)
fight_results_data.loc[:,'event_name'] = fight_results_data.event_name.astype(str)


#To date time conversion
event_data['event_date'] = pd.to_datetime(event_data.event_date)
fighter_details['DOB'] = pd.to_datetime(fighter_details.DOB)
fight_results_data['date'] = pd.to_datetime(fight_results_data.date)


In [52]:
# Create wins and losses columns
record_table.loc[:, "Wins"] = record_table["Record"].str.split(r"[-–]").str[0]
record_table.loc[:, "Losses"] = record_table["Record"].str.split(r"[-–]").str[1]

# Convert columns to numeric
record_table["Wins"] = pd.to_numeric(record_table["Wins"], errors="coerce")
record_table.loc[:, "Losses"] = pd.to_numeric(
    record_table["Losses"].str.replace(r"\s*\(\d+\)", "", regex=True), errors="coerce"
)


In [53]:
def convert_to_ymd(date_col):
    """
    Converts multiple date formats ('Month Day, Year' and 'Day Month Year') into 'YYYY-MM-DD'.
    Uses 'mixed' format to auto-detect formats in each row.
    """

    date_col = date_col.str.replace(r"\(.*?\)", "", regex=True).str.strip()

    return pd.to_datetime(date_col, format="mixed", errors="coerce").dt.strftime(
        "%Y-%m-%d"
    )

# Convert the date column
record_table["Date"] = convert_to_ymd(record_table["Date"])

#### Adjusting Win/Loss Columns to Prevent Information Leakage

The *wins* and *losses* columns already include the outcome of the fight on that date. To ensure a clean training dataset and avoid information leakage, we need to adjust these columns accordingly. If the fighter lost the fight, we must deduct 1 from the *losses* column. If the fighter won, we must deduct 1 from the *wins* column.

In [54]:
# Define patterns for wins and losses
win_patterns = re.compile(r'^(win|won|winn|wi)$', re.IGNORECASE)
loss_patterns = re.compile(r'^(loss|losa|:loss)$', re.IGNORECASE)

# Generalized function to clean results
def clean_result(result):
    if isinstance(result,str):
        result_cleaned = result.strip().lower()
        if re.match(win_patterns, result_cleaned):
            return 'Win'
        elif re.match(loss_patterns, result_cleaned):
            return 'Loss'
        else:
            return None  # Drop NC, Draw, and any other irrelevant entries

# Apply the cleaning function
record_table['Res.'] = record_table['Res.'].apply(clean_result)

In [55]:
record_table.loc[record_table['Res.'] == 'Loss', 'Losses'] +=  - 1
record_table.loc[record_table['Res.'] == 'Win', 'Wins'] +=  - 1

In [56]:
# Create win percentage column
record_table["Win %"] = record_table.apply(
    lambda x: x["Wins"] / (x["Wins"] + x["Losses"]) if x["Wins"] != 0 else 0, axis=1
)

In [57]:
def win_streak_(df):

    df.sort_values("date", inplace=True)
    win_streak = 0
    win_streaks = []

    def get_result(row):

        for fighter in [row["fighter1"], row["fighter2"]]:
            if fighter == row["winner"]:
                return "Win"
            elif fighter == row["loser"]:
                return "Loss"
            else:
                return None

    return win_streaks

In [58]:
def calculate_win_streak_vectorized(df):
    df = df.sort_values(by=["Fighter", "Date"])  # Ensure chronological order

    # Group by Fighter to calculate win streaks within each fighter's history
    def compute_streak(group):
        # Compare Wins with previous row to detect winning streaks
        win_diff = group["Wins"].diff().fillna(0) > 0
        
        # Create a streak ID that increments when a streak resets
        streak_id = (~win_diff).cumsum()

        # Compute the streak length using cumulative count within each streak group
        group["Win Streak"] = group.groupby(streak_id).cumcount() + 1
        return group

    return df.groupby("Fighter", group_keys=False).apply(compute_streak)

# Apply the vectorized function
record_table = calculate_win_streak_vectorized(record_table)

/tmp/ipykernel_21969/159109184.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Fighter", group_keys=False).apply(compute_streak)


In [59]:
record_table['Method'] = record_table['Method'].str.split(" ").str[0]

method_mapper = {
    "Decision": "DEC",
    "Decision(unanimous)": "DEC",
    "Submission": "SUB",
    "TKO": "KOTKO",
    "KO": "KOTKO",
    "KO/TKO": "KOTKO",
    "KO(Punches)": "KOTKO",
    "Technical": "Other",
    "Draw": "Other",
    "NC": "Other",
    "DQ": "Other",
    "No": "Other",
    "Verbal": "Other",
    "Unknown": "Other",
    "Disqualification": "DIS"
}

record_table['Method'] = record_table['Method'].map(method_mapper)

table =  record_table.copy()


method_columns = [
    "win_by_DEC",
    "win_by_KOTKO",
    "win_by_SUB",
    "win_by_Other",
    "loss_by_DEC",
    "loss_by_DIS",
    "loss_by_Other",
    "loss_by_SUB",
    "loss_by_KOTKO",
]

for column in method_columns:
    table[column] = 0

for column in method_columns:
    method_value = "_".join(column.split("_")[2:])  # Extract KO_TKO, SUB, etc.

    # Determine if it's a win or loss column
    if column.startswith("win_by"):
        table.loc[(table['Res.'] == 'Win') & (table['Method'] == method_value), column] = 1

    elif column.startswith("loss_by"):
        table.loc[(table['Res.'] == 'Loss') & (table['Method'] == method_value), column] = 1

In [60]:
# sort the table
table = table.sort_values(by=["Fighter", "Date"], ascending=[True, True]).reset_index(drop=True)


In [61]:

# Compute cumulative sum correctly while keeping Fighter in the table
cum_sum_table = (
    table.groupby("Fighter")[method_columns]  # Group by Fighter
    .cumsum()  # Compute cumulative sum
    .reset_index()  # Drop old index after apply
    .shift(1)  # Shift the cumulative sum by 1 to avoid including the current fight
)

# Restore "Fighter" and "Date"
cum_sum_table["Fighter"] = table["Fighter"]
cum_sum_table["Date"] = table["Date"]

# Ensure "Fighter" and "Date" are the first two columns
column_order = ["Fighter", "Date"] + [col for col in cum_sum_table.columns if col not in ["Fighter", "Date"]]
cum_sum_table = cum_sum_table[column_order]

# First fight for each fighter should be 0
cum_sum_table.loc[table.groupby("Fighter").head(1).index, method_columns] = 0

# Rmoeve column called index 
cum_sum_table = cum_sum_table.drop(columns=['index'])


In [62]:
# Join the cumulative sum table with the original table
record_table = pd.merge(record_table,cum_sum_table, on=["Fighter", "Date"], how="left")

cum_sum

In [63]:
# create win streak column
record_table.loc[:, "Win Streak"] = 0
record_table.loc[:, "Win Streak"] = record_table["Win Streak"].astype(int)

### Goal: Creating a Proper Match Dataset

The following code blocks aim to structure a clean and reliable match dataset by:

1. **Generating unique keys** for different fight-stat tables, enabling data joins.
2. **Assigning two fighter-specific keys** per match—one for each fighter—so we can later join their respective fight stats.
3. **Removing duplicate matches** by selecting only unique fight keys, preventing redundancy and avoiding information leakage.


In [64]:
# Create unique keys for joining later using the url fragments
event_data['event_url_fragment'] = (
    event_data['event_url'].str.split("/").str[-1]
)

event_card_data['fight_url_fragment'] = (
    event_card_data['fight_url'].str.split("/").str[-1]
)

fight_details_data['event_url_fragment'] = (
    fight_details_data['event_url'].str.split("/").str[-1]
)

fight_details_data['fighter_url_fragment'] = (
    fight_details_data['fighter_url'].str.split("/").str[-1]
)

fighter_details["fighter_url_fragment"] = (
    fighter_details["fighter_url"].str.split("/").str[-1]
)

fight_results_data["fight_url_fragment"] = (
    fight_results_data["fight_url"].str.split("/").str[-1]
)

fight_results_data["fighter_url_fragment"] = (
    fight_results_data["fighter_url"].str.split("/").str[-1]
)

In [65]:
# randomize the data and drop duplicate fights
fight_results_random = fight_results_data.sample(frac=1)

distinct_fight_result = fight_results_random.drop_duplicates(subset="fight_url_fragment")

# Req columns
fight_results_req_col = distinct_fight_result[
    [
        "date",
        "event_name",
        "fight_url_fragment",
        "fighter_url_fragment",
        "fighter_name",
        "opponent_name",
        "round",
        "win_loss",
    ]
]

In [66]:
# 
unique_fighter_keys = fight_results_data.drop_duplicates(
    subset=["fighter_name", "fighter_url_fragment", "date"]
)

# merge the fight results with the fight details
fight_results = pd.merge(
    fight_results_req_col,
    unique_fighter_keys[
        ["fighter_name", "fighter_url_fragment", "date"]
    ],
    how="left",
    left_on=["opponent_name", "date"],
    right_on=["fighter_name", "date"],
    suffixes=("_a", "_b"),
)

# Create a result column
fight_results["result"] = fight_results["win_loss"].apply(
    lambda x: "a" if x == "win" else ("b" if x == "loss" else x)
)

In [67]:
fight_results = fight_results[
    [
        "date",
        "event_name",
        "fight_url_fragment",
        "fighter_url_fragment_a",
        "fighter_url_fragment_b",
        "fighter_name_a",
        "fighter_name_b",
        "round",
        "result",
    ]
]

fight_results.shape


(10152, 9)

In [68]:
# Join weight classes 
fight_results = fight_results.merge(
    event_card_data[["fight_url_fragment", "weight_class"]],
    how="left",
    on="fight_url_fragment",
)

print(fight_results.shape, fight_results.columns)

(10152, 10) Index(['date', 'event_name', 'fight_url_fragment', 'fighter_url_fragment_a',
       'fighter_url_fragment_b', 'fighter_name_a', 'fighter_name_b', 'round',
       'result', 'weight_class'],
      dtype='object')


### Creating the Fight Stats Table  

The following steps are performed to structure the fight stats table:

1. **Splitting action columns** formatted as `"<value1> of <value2>"` into separate columns:  
   - `value1` represents successful actions.  
   - `value2` represents attempted actions.  

2. **Calculating rolling averages** for all actions:  
   - Computes the average of previous actions for each round across past fights leading up to the current fight.  

3. **Joining the processed data** on the match key to integrate fight statistics effectively.  


In [69]:
actions_columns = [
    "sign_str",
    "total_str",
    "td",
    "head_str",
    "body_str",
    "leg_str",
    "distance",
    "clinch",
    "ground",
]

fight_details_data[actions_columns].head()

,sign_str,total_str,td,head_str,body_str,leg_str,distance,clinch,ground
0,13 of 19,68 of 89,1 of 1,11 of 16,1 of 2,1 of 1,0 of 1,0 of 0,13 of 18
1,0 of 2,35 of 49,0 of 0,0 of 1,0 of 1,0 of 0,0 of 2,0 of 0,0 of 0
2,4 of 8,53 of 64,1 of 1,4 of 8,0 of 0,0 of 0,0 of 1,0 of 0,4 of 7
3,2 of 6,27 of 41,0 of 0,2 of 5,0 of 0,0 of 1,1 of 5,0 of 0,1 of 1
4,1 of 4,1 of 4,0 of 0,0 of 3,1 of 1,0 of 0,0 of 2,1 of 2,0 of 0


In [70]:
def split_attempts_landed(data, col_names):
    for col_name in col_names:
        # Split the column and assign it back
        split_cols = (
            data[col_name]
            .str.split(" of ", expand=True)
            .rename(columns={0: col_name + "_successful", 1: col_name + "_attempts"})
        )

        # Convert to integers
        split_cols[col_name + "_successful"] = split_cols[
            col_name + "_successful"
        ].astype(int)
        split_cols[col_name + "_attempts"] = split_cols[col_name + "_attempts"].astype(
            int
        )

        # Join with the original DataFrame
        data = data.drop(columns=[col_name])  # Remove original column
        data = data.join(split_cols)  # Add the new columns

    return data

fight_stats = split_attempts_landed(fight_details_data, actions_columns)

In [71]:
def time_to_seconds(time_str):
    if isinstance(time_str, str) and ":" in time_str:
        minutes, seconds = map(int, time_str.split(":"))
        return minutes * 60 + seconds
    return None  # Assign None for missing or invalid values


# Apply conversion to the column
fight_stats["ctrl_time_sec"] = fight_stats["ctrl_time"].apply(time_to_seconds)

In [72]:
attempts_successful_col = [
    x
    for x in fight_stats.columns
    if re.search(r"attempts", x) or re.search(r"successful", x)
]

remaining_col = ["kd", "ctrl_time_sec", "sub_att"]

In [73]:
stats_columns = attempts_successful_col + remaining_col
fight_stats.columns

Index(['type', 'fighter', 'fighter_url', 'event_url', 'fight_url', 'round',
       'kd', 'sign_str_pct', 'td_pct', 'sub_att', 'rev', 'ctrl_time',
       'event_url_fragment', 'fighter_url_fragment', 'sign_str_successful',
       'sign_str_attempts', 'total_str_successful', 'total_str_attempts',
       'td_successful', 'td_attempts', 'head_str_successful',
       'head_str_attempts', 'body_str_successful', 'body_str_attempts',
       'leg_str_successful', 'leg_str_attempts', 'distance_successful',
       'distance_attempts', 'clinch_successful', 'clinch_attempts',
       'ground_successful', 'ground_attempts', 'ctrl_time_sec'],
      dtype='object')

In [74]:
# join event data
complete_fight_stats = pd.merge(
    fight_stats,
    event_data[["event_name", "event_url_fragment", "event_date"]],
    how="left",
    on="event_url_fragment",
)

# sort the data
complete_fight_stats = complete_fight_stats.sort_values(
    by=["fighter", "event_date", "round"]
).reset_index(drop=True)


complete_fight_stats.head()

,type,fighter,fighter_url,event_url,fight_url,round,kd,sign_str_pct,td_pct,sub_att,...,leg_str_attempts,distance_successful,distance_attempts,clinch_successful,clinch_attempts,ground_successful,ground_attempts,ctrl_time_sec,event_name,event_date
0,fight_details,Alatengheili,http://www.ufcstats.com/fighter-details/1897b7...,http://www.ufcstats.com/event-details/2c104b7e...,http://www.ufcstats.com/fight-details/2e001d8c...,1,0,43%,---,0,...,0,14,32,0,0,0,0,0.0,UFC Fight Night: Andrade vs. Zhang,2019-08-31
1,fight_details,Alatengheili,http://www.ufcstats.com/fighter-details/1897b7...,http://www.ufcstats.com/event-details/2c104b7e...,http://www.ufcstats.com/fight-details/2e001d8c...,2,0,35%,---,0,...,0,11,31,0,0,0,0,0.0,UFC Fight Night: Andrade vs. Zhang,2019-08-31
2,fight_details,Alatengheili,http://www.ufcstats.com/fighter-details/1897b7...,http://www.ufcstats.com/event-details/74fefd43...,http://www.ufcstats.com/fight-details/15aa55a9...,1,0,24%,0%,0,...,2,12,50,0,0,0,0,15.0,UFC Fight Night: Edgar vs. The Korean Zombie,2019-12-21
3,fight_details,Alatengheili,http://www.ufcstats.com/fighter-details/1897b7...,http://www.ufcstats.com/event-details/74fefd43...,http://www.ufcstats.com/fight-details/15aa55a9...,2,0,43%,---,0,...,6,28,65,0,0,0,0,0.0,UFC Fight Night: Edgar vs. The Korean Zombie,2019-12-21
4,fight_details,Alatengheili,http://www.ufcstats.com/fighter-details/1897b7...,http://www.ufcstats.com/event-details/805ad180...,http://www.ufcstats.com/fight-details/ac6b2a5f...,1,0,29%,---,0,...,5,19,64,0,0,0,0,0.0,UFC Fight Night: Holm vs. Aldana,2020-10-03


In [75]:
for col in stats_columns:
    complete_fight_stats[col] = pd.to_numeric(complete_fight_stats[col], errors="coerce")




In [76]:
# Compute rolling averages
rolling_avg_df = (
    complete_fight_stats.groupby(
        ["fighter","fighter_url_fragment"])
        [stats_columns]
    .expanding()
    .mean()
    .round(2)
    .reset_index()

)

rolling_avg_df = rolling_avg_df.rename(columns = {'level_2':'event_date'})

rolling_avg_df['event_date'] = complete_fight_stats['event_date']


In [77]:
# Get the last value for every fighter round 
last_round_stats = (
    rolling_avg_df.groupby(
        ["fighter","fighter_url_fragment","event_date"])
        [stats_columns]
    .last()
    .shift(1)
    .reset_index()
)

last_round_stats['row_number'] = (
    last_round_stats.groupby(["fighter"]).cumcount()
)


#Replace the first row stats column with n/a
if last_round_stats['row_number'].min() == 0:
    last_round_stats.loc[last_round_stats['row_number'] == 0, stats_columns] = None

Join each fighter's stats to the match data using the event date as the key, ensures that the fighter's historical performance is correctly aligned with all matches prior to their respective fights.

In [78]:
# Merge the rolling averages with the last round stats
complete_fight_stats_a = last_round_stats.copy()
complete_fight_stats_b = last_round_stats.copy()

# Rename columns
complete_fight_stats_a.columns = [x + "_a" for x in complete_fight_stats_a.columns]
complete_fight_stats_b.columns = [x + "_b" for x in complete_fight_stats_b.columns]

fight_results['date'] = pd.to_datetime(fight_results['date'])


# Join fighter stats data on result data
merge_fighter_a_stats = pd.merge(
    fight_results,
    complete_fight_stats_a,

    how="left",
    left_on=["fighter_url_fragment_a", "date"],
    right_on=["fighter_url_fragment_a", "event_date_a"],

)

merge_fighter_a_stats = merge_fighter_a_stats[

    [x for x in merge_fighter_a_stats.columns if x != "fighter_url_fragment"]

]

merge_fighter_b_stats = pd.merge(
    merge_fighter_a_stats,
    complete_fight_stats_b,

    how="left",
    left_on=["fighter_url_fragment_b", "date"],
    right_on=["fighter_url_fragment_b", "event_date_b"],
    suffixes=("", "_b"),

)

fight_stats_data = merge_fighter_b_stats

In [79]:
fight_stats_data.columns

Index(['date', 'event_name', 'fight_url_fragment', 'fighter_url_fragment_a',
       'fighter_url_fragment_b', 'fighter_name_a', 'fighter_name_b', 'round',
       'result', 'weight_class', 'fighter_a', 'event_date_a',
       'sign_str_successful_a', 'sign_str_attempts_a',
       'total_str_successful_a', 'total_str_attempts_a', 'td_successful_a',
       'td_attempts_a', 'head_str_successful_a', 'head_str_attempts_a',
       'body_str_successful_a', 'body_str_attempts_a', 'leg_str_successful_a',
       'leg_str_attempts_a', 'distance_successful_a', 'distance_attempts_a',
       'clinch_successful_a', 'clinch_attempts_a', 'ground_successful_a',
       'ground_attempts_a', 'kd_a', 'ctrl_time_sec_a', 'sub_att_a',
       'row_number_a', 'fighter_b', 'event_date_b', 'sign_str_successful_b',
       'sign_str_attempts_b', 'total_str_successful_b', 'total_str_attempts_b',
       'td_successful_b', 'td_attempts_b', 'head_str_successful_b',
       'head_str_attempts_b', 'body_str_successful_b', 'b

In [80]:
fighter_details.head()

,type,fighter_url,fighter_name,height,weight,reach,stance,DOB,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,fighter_key,fighter_url_fragment
0,fighter_details,http://www.ufcstats.com/fighter-details/2997e7...,Wang Cong,"5' 6""",125 lbs.,"66""",Southpaw,1992-05-15,6.38,52%,1.86,61%,1.61,75%,100%,1.1,7.561992e+10,2997e7fe3c9d3d4a
1,fighter_details,http://www.ufcstats.com/fighter-details/d5ae80...,Katsuhiko Nagata,"5' 7""",154 lbs.,--,Southpaw,1973-10-31,1.05,19%,1.50,63%,0.98,28%,28%,0.0,6.051973e+10,d5ae8074631762fc
2,fighter_details,http://www.ufcstats.com/fighter-details/7be328...,Yukiya Naito,"6' 1""",205 lbs.,--,Orthodox,NaT,1.54,33%,5.13,60%,0.00,0%,0%,0.0,NaN,7be328bb3c2ea22e
3,fighter_details,http://www.ufcstats.com/fighter-details/749685...,Remco Pardoel,"6' 3""",260 lbs.,--,Southpaw,NaT,0.00,0%,0.00,0%,0.00,0%,0%,0.0,NaN,749685d24e2cac50
4,fighter_details,http://www.ufcstats.com/fighter-details/14ff46...,Norman Parke,"5' 11""",155 lbs.,"70""",Southpaw,1986-12-22,3.06,35%,2.88,64%,1.98,26%,77%,0.0,1.911986e+10,14ff46596ad80609


### Creating additional features  

Enhance the dataset by incorporating the following features:  

1. **Add height, reach and Age** – Age calculated by subtracting the fighter's date of birth (DOB) from the event date. 
2. **Wins/Losses Data** – Join Win and Loss data computed from the Wikipedia record table.  
3. **Win Ratio**

**How:** 

For 1. fighter_details table is join the compute the age

For 2. Data from the wikipedia table is joined. 
- No existing corresponding keys exists. 
- A combination of string matching techniques is used to join the data. 
      

In [81]:
fighter_details = fighter_details[["fighter_url_fragment", "height", "reach", "DOB"]]

# Transfrom the height and reach columns to cm
def inch_to_cm(value):
    # Check if the value is in feet and inches (e.g., 5' 10")
    if re.match(r"\d+'\s*\d+\"", value):
        feet, inches = re.findall(r'\d+', value)
        total_inches = int(feet) * 12 + int(inches)
    # Check if the value is in inches only (e.g., 73")
    elif re.match(r"\d+\"", value):
        total_inches = int(re.findall(r'\d+', value)[0])
    else:
        return None  # Handle unexpected formats
    return round(total_inches * 2.54, 2)

for column in ["height", "reach"]:
    fighter_details[column] = fighter_details[column].apply(inch_to_cm)

In [82]:
fighter_details_a = fighter_details.copy()
fighter_details_b = fighter_details.copy()

fighter_details_a.columns = [x + "_a" for x in fighter_details_a.columns]
fighter_details_b.columns = [x + "_b" for x in fighter_details_b.columns]

In [83]:
fighter_stats_detail_a = pd.merge(
    fight_stats_data,
    fighter_details_a,
    how="left",
    left_on="fighter_url_fragment_a",
    right_on="fighter_url_fragment_a",
    suffixes=("", "_a"),
)

fighter_stats_detail_b = pd.merge(
    fighter_stats_detail_a,
    fighter_details_b,
    how="left",
    left_on="fighter_url_fragment_b",
    right_on="fighter_url_fragment_b",
    suffixes=("", "_b"),
)
fighter_stats_detail = fighter_stats_detail_b

In [84]:
# Compute ages in years
fighter_stats_detail["age_a"] = (
    fighter_stats_detail["event_date_a"] - fighter_stats_detail["DOB_a"]
).dt.days // 365
fighter_stats_detail["age_b"] = (
    fighter_stats_detail["event_date_a"] - fighter_stats_detail["DOB_b"]
).dt.days // 365

## Creating Compatibility Between Record Table and Fight Stats

To merge the `record_table` (Wikipedia data) and the `fight_stats` (UFC stats website data), we need to ensure compatibility. The `fight_result` dataset from (UFC stats website) is used to add fighter keys to the `record_table`(Wikipedia data) to do this:

1. **Cross Join by Date**  
   - We generate all possible combinations of records from both tables where the event dates match.

2. **Event Name Matching using Token Set Ratio**  
   - We apply a string matching technique (`token_set_ratio`) to compare the `event_name` column from both sources.  
   - Only records where the similarity score is greater than **95** are retained.

3. **Fighter-Opponent Name Matching**  
   - We create a concatenated string by joining `fighter_name` and `opponent_name` for both datasets.  
   - Example: `"Fighter A vs Fighter B"` should be similar across both datasets.

4. **Filtering by Similarity Ratio**  
   - We apply another string similarity check (`similarity ratio`) to compare the fighter-opponent strings.  
   - Only records where the similarity score is greater than **75** are kept.

This ensures that the final dataset consists of properly matched fight records from both sources.


In [85]:
# Merge the record table with the fighter stats detail
req_columns = [
    "Fighter",
    "Opponent",
    "Record",
    "Date",
    "Wins",
    "Losses",
    "Event",
    "Win %",
    "Win Streak",
    "win_by_DEC",
    "win_by_KOTKO",
    "win_by_SUB",
    "win_by_Other",
    "loss_by_DEC",
    "loss_by_DIS",
    "loss_by_Other",
    "loss_by_SUB",
    "loss_by_KOTKO",
]
record_table = record_table[req_columns]

In [86]:
# Convert the date columns to datetime
fight_results_data["date"] = pd.to_datetime(fight_results_data["date"])
record_table["Date"] = pd.to_datetime(record_table["Date"])

# Create possible date ranges as lists
fight_results_data["possible_date"] = fight_results_data["date"].apply(
    lambda x: [x, x + pd.DateOffset(days=1), x - pd.DateOffset(days=1)]
)

record_table["possible_date"] = record_table["Date"].apply(
    lambda x: [x, x + pd.DateOffset(days=1), x - pd.DateOffset(days=1)]
)

# Explode to create separate rows for each possible date
fight_results_exploded = fight_results_data.explode("possible_date")
record_table_exploded = record_table.explode("possible_date")


# transform event to lower case 
fight_results_exploded['event_name'] = fight_results_exploded['event_name'].str.lower()
record_table_exploded['Event'] = record_table_exploded['Event'].str.lower()

# Merge on the exploded date columns
explode_fight_results = pd.merge(
    fight_results_exploded,
    record_table_exploded,
    how="left",
    left_on="possible_date",
    right_on="possible_date"
)
print(
    "fight results shape: ",
    fight_results_data.shape,
    "\n",
    "record_table shape: ",
    record_table.shape,
    "\n",
    "new table shape: ",
    explode_fight_results.shape,
)

fight results shape:  (19769, 22) 
 record_table shape:  (92482, 19) 
 new table shape:  (4372786, 40)


In [87]:

import gc
gc.collect()

del fight_results_exploded
del record_table_exploded
del fight_results_data
del fight_details_data
del record_table

explode_fight_results["event_ratio"] = explode_fight_results.apply(
    lambda x: fuzz.token_set_ratio(x["Event"], x["event_name"]), axis=1
)

In [88]:
fight_result_ratio_95 = explode_fight_results[
    explode_fight_results["event_ratio"] >= 95
]

In [89]:
#
def create_temp_keys(df, columns_fight_stats, columns_wiki):

    df = df.copy()

    df.loc[:,"temp_fight_stats_key"] = (
        df[columns_fight_stats].astype(str).apply(lambda x: " ".join(x), axis=1)
    )

    df.loc[:,"temp_wiki_key"] = (
        df[columns_wiki].astype(str).apply(lambda x: " ".join(x), axis=1)
    )
    return df

def calculate_ratio_similarity(df, value_a, value_b):
    """Compute Jaccard similarity between two columns of a DataFrame (word-based)."""
    ratios = []

    df = df.copy()

    for a, b in zip(df[value_a], df[value_b]):
        ratio = fuzz.ratio(a, b)
        ratios.append(ratio)

    return ratios

fight_result_ratio_95 = create_temp_keys(
    fight_result_ratio_95,
    ["fighter_name", "opponent_name"],
    ["Fighter", "Opponent"],
)

fight_result_ratio_95.loc[:,"fighter_ratio"] = calculate_ratio_similarity(
    fight_result_ratio_95, "temp_fight_stats_key", "temp_wiki_key"
)


In [90]:
fight_result_fighter_ratio_75 = fight_result_ratio_95[
    fight_result_ratio_95.fighter_ratio >= 75
]

fight_result_fighter_ratio_75.head()

,type,fighter_url,fight_url,event_url,DOB,fighter_name,opponent_name,KD,STR,TD,...,win_by_Other,loss_by_DEC,loss_by_DIS,loss_by_Other,loss_by_SUB,loss_by_KOTKO,event_ratio,temp_fight_stats_key,temp_wiki_key,fighter_ratio
0,fight_results,http://www.ufcstats.com/fighter-details/24656e...,http://www.ufcstats.com/fight-details/b989d4d7...,http://www.ufcstats.com/event-details/3ab0df1d...,1987-11-19,Aiemann Zahabi,Pedro Munhoz,0/0,109/95,0/0,...,0.0,1.0,0.0,0.0,0.0,1.0,100,Aiemann Zahabi Pedro Munhoz,Aiemann Zahabi Pedro Munhoz,100
65,fight_results,http://www.ufcstats.com/fighter-details/24656e...,http://www.ufcstats.com/fight-details/b989d4d7...,http://www.ufcstats.com/event-details/3ab0df1d...,1987-11-19,Aiemann Zahabi,Pedro Munhoz,0/0,109/95,0/0,...,0.0,1.0,0.0,0.0,0.0,1.0,100,Aiemann Zahabi Pedro Munhoz,Aiemann Zahabi Pedro Munhoz,100
122,fight_results,http://www.ufcstats.com/fighter-details/24656e...,http://www.ufcstats.com/fight-details/b989d4d7...,http://www.ufcstats.com/event-details/3ab0df1d...,1987-11-19,Aiemann Zahabi,Pedro Munhoz,0/0,109/95,0/0,...,0.0,1.0,0.0,0.0,0.0,1.0,100,Aiemann Zahabi Pedro Munhoz,Aiemann Zahabi Pedro Munhoz,100
187,fight_results,http://www.ufcstats.com/fighter-details/24656e...,http://www.ufcstats.com/fight-details/a7d397cd...,http://www.ufcstats.com/event-details/e4a9dbad...,1987-11-19,Aiemann Zahabi,Javid Basharat,0/0,75/67,0/0,...,0.0,1.0,0.0,0.0,0.0,1.0,100,Aiemann Zahabi Javid Basharat,Aiemann Zahabi Javid Basharat,100
263,fight_results,http://www.ufcstats.com/fighter-details/24656e...,http://www.ufcstats.com/fight-details/a7d397cd...,http://www.ufcstats.com/event-details/e4a9dbad...,1987-11-19,Aiemann Zahabi,Javid Basharat,0/0,75/67,0/0,...,0.0,1.0,0.0,0.0,0.0,1.0,100,Aiemann Zahabi Javid Basharat,Aiemann Zahabi Javid Basharat,100


In [91]:
# In case of duplicate we will take the the value with the highest ratio
def assign_rank(df):
    """Assign a rank column for each unique (date, Fighter, Opponent) group based on fighter_ratio."""
    df_sorted = df.sort_values(
        by=["date", "Fighter", "Opponent", "fighter_ratio"],
        ascending=[True, True, True, False],
    )
    df_sorted["rank"] = (
        df_sorted.groupby(["date", "Fighter", "Opponent"]).cumcount() + 1
    )
    return df_sorted


# Assign ranks
fight_result_fighter_ratio_75_ranked = assign_rank(fight_result_fighter_ratio_75)

fight_w_record = fight_result_fighter_ratio_75_ranked[
    fight_result_fighter_ratio_75_ranked["rank"] == 1
]


fight_w_record.columns

Index(['type', 'fighter_url', 'fight_url', 'event_url', 'DOB', 'fighter_name',
       'opponent_name', 'KD', 'STR', 'TD', 'SUB', 'event_name', 'date',
       'method', 'round', 'time', 'win_loss', 'fighter_key', 'event_key',
       'fight_url_fragment', 'fighter_url_fragment', 'possible_date',
       'Fighter', 'Opponent', 'Record', 'Date', 'Wins', 'Losses', 'Event',
       'Win %', 'Win Streak', 'win_by_DEC', 'win_by_KOTKO', 'win_by_SUB',
       'win_by_Other', 'loss_by_DEC', 'loss_by_DIS', 'loss_by_Other',
       'loss_by_SUB', 'loss_by_KOTKO', 'event_ratio', 'temp_fight_stats_key',
       'temp_wiki_key', 'fighter_ratio', 'rank'],
      dtype='object')

In [92]:
req_columns = [
    "date",
    "fighter_url_fragment",
    "Wins",
    "Losses",
    "Win %",
    "Win Streak",
    "win_by_DEC",
    "win_by_KOTKO",
    "win_by_SUB",
    "win_by_Other",
    "loss_by_DEC",
    "loss_by_DIS",
    "loss_by_Other",
    "loss_by_SUB",
    "loss_by_KOTKO",
]
fight_w_l = fight_w_record[req_columns]

Now that we have a **fighter key** for the Wikipedia dataset, we can use it to join it with the UFC fighter stats dataset and finish the training dataset



In [93]:
#  Join the data

fight_w_l_a = fight_w_l.copy()
fight_w_l_b = fight_w_l.copy()

fight_w_l_a.columns = [x + "_a" for x in fight_w_l_a.columns]
fight_w_l_b.columns = [x + "_b" for x in fight_w_l_b.columns]


training_data = pd.merge(
    fighter_stats_detail,
    fight_w_l_a,
    how="left",
    left_on=["fighter_url_fragment_a", "date"],
    right_on=["fighter_url_fragment_a", "date_a"],
    suffixes=("", "_a"),
)

training_data = pd.merge(
    training_data,
    fight_w_l_b,
    how="left",
    left_on=["fighter_url_fragment_b", "date"],
    right_on=["fighter_url_fragment_b", "date_b"],
    suffixes=("", "_b"),
)


column_not_req = ['type_y_a','type_y_b', 'type.']



In [94]:
# Drop Redundant columns
redundant_columns = [    
    "DOB_b",
    'DOB_a',
    "row_number_a",
    "row_number_b",
    "round",
    "fighter_url_fragment_a",
    "fighter_url_fragment_b",
    "date",
    "event_name",
    "row_number_a",
    "row_number_b",
    "date_a",
    "date_b",
    "fighter_a",
    "fighter_b",
    "fight_url_fragment",

]

training_data = training_data.drop(columns=redundant_columns)


training_data.to_csv(get_file_path("collection/transformed", "training_data.csv"), index=False)
